##Torrent To Dropbox Downloader

##Install libtorrent and Initialize Session

In [ ]:
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

In [ ]:
!wget https://downloads.rclone.org/v1.53.2/rclone-v1.53.2-linux-amd64.deb
!apt install ./rclone-v1.53.2-linux-amd64.deb

##Mount GDrive common

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

##Rcolon config Dropbox

In [ ]:
!rclone config

##Dropbox mount

In [ ]:
!sudo mkdir /content/dropbox
!nohup rclone --vfs-cache-mode writes mount dropbox: /content/dropbox &

{"access_token":"sl.CDlA00HEr5LeDRO_1mXZJBe5v0KX_nusgz880-IXeb11a4_Nusr3rYa6dFbYOxXd-XPVY5jXJ0PywoKhdLnJoHWMjrDu8sxKevl5UCVmnuXVY87uUD-cYfF56GWIqW67bJfdki2Ts5OCysk","token_type":"bearer","refresh_token":"9D3yJe8dKukAAAAAAAAAAdwNJxLyrLVAvdoLiF8opI4rRCESC7heY1ihn7iOHXZK","expiry":"2024-12-31T00:03:49.8562854+06:00"}

##Add From Torrent File

In [ ]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

##Start Download

In [ ]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)